In [3]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper

In [ ]:
arxivwrap=ArxivAPIWrapper(top_k_results=2,dox_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrappper=arxivwrap,description=" Query arxiv papers")

"Published: 2018-07-02\nTitle: The Matrix Calculus You Need For Deep Learning\nAuthors: Terence Parr, Jeremy Howard\nSummary: This paper is an attempt to explain all the matrix calculus you need in order\nto understand the training of deep neural networks. We assume no math knowledge\nbeyond what you learned in calculus 1, and provide links to help you refresh\nthe necessary math where needed. Note that you do not need to understand this\nmaterial before you start learning to train and use deep learning in practice;\nrather, this material is for those who are already familiar with the basics of\nneural networks, and wish to deepen their understanding of the underlying math.\nDon't worry if you get stuck at some point along the way---just go back and\nreread the previous section, and try writing down and working through some\nexamples. And if you're still stuck, we're happy to answer your questions in\nthe Theory category at forums.fast.ai. Note: There is a reference section at\nthe end

In [13]:
wikiwrap=WikipediaAPIWrapper(top_k_results=1,dox_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=wikiwrap)


In [15]:
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


In [16]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily=TavilySearchResults()


In [51]:
tools=[arxiv,wiki,tavily]
from langchain_groq import ChatGroq
llm=ChatGroq(model="qwen-qwq-32b",  max_tokens=300 )
# llm.invoke('what is ai')
llm_tool=llm.bind_tools(tools=tools)



In [47]:
from typing import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages


In [53]:
class State(TypedDict):
      messages:Annotated[list[AnyMessage],add_messages] 
from IPython.display import Image,display
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
def tool_llm(state:State):
      return {"messages":[llm_tool.invoke(state["messages"])]}
builder=StateGraph(State)
builder.add_node("tool_llm",tool_llm)
builder.add_node("tools",ToolNode(tools))
builder.add_edge(START,"tool_llm")
builder.add_conditional_edges("tool_llm",tools_condition)
builder.add_edge("tools","tool_llm")
graph=builder.compile()


In [55]:
messages=graph.invoke({"messages":"what is recent new of mumbai and what is latest research paper in ml"})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

what is recent new of mumbai and what is latest research paper in ml
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (nwdg1a2c8)
 Call ID: nwdg1a2c8
  Args:
    query: recent news Mumbai
  arxiv (achcdvy1x)
 Call ID: achcdvy1x
  Args:
    query: latest machine learning research papers
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Mumbai news today, latest Mumbai news - The Hindu", "url": "https://www.thehindu.com/news/cities/mumbai/", "content": "The Bombay High Court on Thursday (July 10, 2025) upheld the government decision to rehabilitate Dharavi redevelopment project affected people (PAP) on the salt pan land, dismissing the Public Interest Litigation (PIL) filed by a Mumbai-based lawyer and social activist. upheld the government decision to rehabil